In [ ]:
 # imports
# !pip install spacy==2.3.5
# !python -m spacy download en_core_web_sm
# !python -m spacy download en
# import spacy


In [3]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import matplotlib.pyplot as plt
import spacy
import operator

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
data = pd.read_csv('reddit_wsb.csv')
data.head()

nyseStocks = pd.read_csv('nyse.csv')
nasdaqStocks = pd.read_csv('nasdaq.csv')

data.head()

FileNotFoundError: ignored

In [ ]:
data['length'] = [len(i.split()) for i in data['title']]
data['length'].min()

4

In [ ]:
#Contains a list of every ticker tape used on the NYSE and NASDAQ
stocks = set(nyseStocks['Symbol'])
stocks.update(set(nasdaqStocks['Symbol']))


{'TPVG', 'HIG', 'BQ', 'PATH', 'NEE', 'MNP', 'WD', 'AXL', 'RENN', 'DTW', 'VCRA', 'LEAP', 'SPH', 'HHLA', 'ELF', 'WU', 'NSTB', 'PAC', 'GJO', 'THC', 'CNHI', 'SPGS', 'DSL', 'SNV^E', 'MGY', 'GM', 'AHT^D', 'CBU', 'APG', 'COF^J', 'AWK', 'DY', 'AVA', 'SRLP', 'DLNG^B', 'EDF', 'MRC', 'DAO', 'TTC', 'EURN', 'TROX', 'CIA', 'CPS', 'SPNT', 'AAT', 'FTAI', 'FLC', 'EARN', 'CURO', 'VTOL', 'CAS', 'HLX', 'OFC', 'FTV', 'SUI', 'CCV', 'SYK', 'RBLX', 'ESTE', 'PEI^B', 'BAX', 'KAR', 'FTAI^B', 'ACR^C', 'LC', 'GAB^J', 'CAH', 'CNO^A', 'SLG^I', 'GNE', 'CAT', 'ALLE', 'BHR^D', 'DCI', 'CFX', 'MHO', 'HSY', 'IAA', 'BKI', 'NPO', 'RQI', 'ASPL', 'FT', 'YEXT', 'HESM', 'BOOT', 'ESS', 'VRTV', 'AMH', 'CAJ', 'KTF', 'FACA', 'HEXO', 'ABR^A', 'MFD', 'TPZ', 'LUMN', 'NETI', 'DTF', 'DSM', 'PFH', 'ACP', 'UNMA', 'USB^R', 'DLB', 'BCS', 'DNP', 'CND', 'JHG', 'FMN', 'EVRI', 'UBP', 'DOV', 'CF', 'KMPR', 'MGR', 'PMT^A', 'CHPT', 'CFG', 'IDT', 'HES', 'DHF', 'SNOW', 'SC', 'FRT^C', 'BSX', 'PKE', 'ADC', 'INGR', 'MFC', 'GMS', 'DS^C', 'DIS', 'GIM', 'R

In [ ]:
data = data[data['length'] > 3].copy()

In [ ]:
data['length'].min()

In [ ]:
def isUppercase(word):
  return word.isupper()

In [ ]:
#RUN ONLY ONCE.  DATA IS IN title_data.csv 

title_data = data[['title','timestamp']].copy()
title_data = title_data.dropna()
nlp = spacy.load("en_core_web_sm")

tickerList = []

for index, row in title_data.iterrows():
  words = nlp(row['title'])
  token_list = [token.orth_ for token in words]
  wordset = set(filter(isUppercase, token_list))
  tempTickers = (wordset.intersection(stocks))
  #print(tempTickers)
  tickerList.append(tempTickers)
  #title_data[index, 'stocks'] = tempTickers
print(tickerList)
title_data['tickers'] = pd.Series(tickerList).values
  
  
  

title_data.title = title_data.title.str.lower()
title_data.title = title_data.title.apply(lambda x:re.sub('@[^\s]+','',x))
title_data.title = title_data.title.apply(lambda x:re.sub(r"http\S+", "", x))
title_data.timestamp = pd.to_datetime(title_data.timestamp).dt.date

[set(), set(), {'GME'}, {'GME', 'NEW', 'CAN', 'FOR'}, {'GME', 'AMC'}, {'GME', 'AN'}, {'GME'}, {'GME', 'NOK', 'AMC'}, set(), {'GME', 'AMC'}, {'GME'}, {'GME', 'AG'}, {'GME'}, set(), {'GME'}, set(), set(), set(), set(), {'GME'}, {'GME'}, set(), set(), {'GME'}, {'GME', 'AMC'}, {'GME'}, set(), {'GME', 'NOK', 'K', 'GO'}, set(), set(), set(), {'GME'}, {'SU', 'PLTR', 'BB', 'GME', 'NOK', 'RIDE'}, set(), {'GME'}, set(), set(), set(), {'GME'}, {'FIZZ'}, {'GME'}, set(), {'GME'}, set(), set(), {'MOD'}, set(), {'GME', 'BB'}, {'GME', 'NOK', 'AMC', 'BB'}, set(), {'AMC'}, set(), {'GME'}, set(), {'RH'}, {'DD'}, set(), {'GME'}, set(), set(), set(), set(), {'GME'}, {'PLTR'}, set(), {'GME'}, set(), set(), set(), set(), set(), {'OR'}, set(), set(), set(), set(), {'GME'}, {'NOK'}, set(), set(), set(), {'GME', 'PT'}, set(), {'AMC'}, {'NOK'}, set(), set(), set(), set(), {'GME'}, {'GME', 'AMC'}, {'GME', 'IT', 'EOD'}, {'GME', 'BIG'}, {'GME'}, set(), {'DD'}, {'AMC'}, {'GME', 'PLTR'}, {'HEAR'}, {'A'}, set(), set()

In [ ]:
#Download the code for the first time after running the prev block

title_data.head()

from google.colab import files

title_data.to_csv('title_data.csv')

files.download("title_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
title_data = pd.read_csv('title_data.csv', parse_dates=['timestamp'])

In [5]:
import nltk
nltk.download('vader_lexicon')

sid = SIA()

title_data['sentiments'] = title_data['title'].apply(lambda x: sid.polarity_scores(' '.join(re.findall(r'\w+',x.lower()))))
title_data['Positive Sentiment'] = title_data['sentiments'].apply(lambda x: x['pos']+1*(10**-6)) 
title_data['Neutral Sentiment'] = title_data['sentiments'].apply(lambda x: x['neu']+1*(10**-6))
title_data['Negative Sentiment'] = title_data['sentiments'].apply(lambda x: x['neg']+1*(10**-6))
title_data.drop(columns=['sentiments'],inplace=True)
title_data.head()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


,Unnamed: 0,title,timestamp,tickers,score,Positive Sentiment,Neutral Sentiment,Negative Sentiment
0,0,"it's not about the money, it's about sending a...",2021-01-28,set(),55,0.000001,1.000001,0.000001
1,1,math professor scott steiner says the numbers ...,2021-01-28,set(),110,0.000001,0.728001,0.272001
2,2,exit the system gme,2021-01-28,{'GME'},0,0.000001,1.000001,0.000001
3,3,new sec filing for gme! can someone less retar...,2021-01-28,"{'GME', 'NEW', 'CAN', 'FOR'}",29,0.138001,0.658001,0.204001
4,4,"not to distract from gme, just thought our amc...",2021-01-28,"{'GME', 'AMC'}",71,0.119001,0.881001,0.000001


In [6]:
title_data['class_name'] = title_data[['Positive Sentiment', 'Neutral Sentiment', 'Negative Sentiment']].idxmax(axis=1)

In [7]:
data = title_data[title_data['class_name'] != 'Neutral Sentiment']
data.reset_index(drop=True, inplace=True)
data.head(5)

,Unnamed: 0,title,timestamp,tickers,score,Positive Sentiment,Neutral Sentiment,Negative Sentiment,class_name
0,14,i love you retards!!!!,2021-01-28,set(),176,0.677001,0.323001,0.000001,Positive Sentiment
1,51,the wallstreetbets war museum,2021-01-28,set(),212,0.000001,0.435001,0.565001,Negative Sentiment
2,101,this took me longer than i'd like to admit. en...,2021-01-28,set(),222,0.517001,0.483001,0.000001,Positive Sentiment
3,114,most recent short interest,2021-01-28,set(),476,0.521001,0.479001,0.000001,Positive Sentiment
4,149,i love the movies: part 1,2021-01-28,{'LOVE'},736,0.583001,0.417001,0.000001,Positive Sentiment


In [8]:
data['class_name'][data['class_name'] == 'Negative Sentiment'] = 0
data['class_name'][data['class_name'] == 'Positive Sentiment'] = 1
data.rename(columns={'class_name':'class'}, inplace=True)

In [9]:
#data = data[['title','class']].copy()
data

,Unnamed: 0,title,timestamp,tickers,score,Positive Sentiment,Neutral Sentiment,Negative Sentiment,class
0,14,i love you retards!!!!,2021-01-28,set(),176,0.677001,0.323001,0.000001,1
1,51,the wallstreetbets war museum,2021-01-28,set(),212,0.000001,0.435001,0.565001,0
2,101,this took me longer than i'd like to admit. en...,2021-01-28,set(),222,0.517001,0.483001,0.000001,1
3,114,most recent short interest,2021-01-28,set(),476,0.521001,0.479001,0.000001,1
4,149,i love the movies: part 1,2021-01-28,{'LOVE'},736,0.583001,0.417001,0.000001,1
...,...,...,...,...,...,...,...,...,...
1848,45350,$10 price target!? i love a good sale!,2021-04-13,set(),532,0.640001,0.360001,0.000001,1
1849,45363,aso gains making my gme losses feel better :d,2021-04-13,"{'GME', 'ASO'}",72,0.408001,0.385001,0.208001,1
1850,45368,i hope tomorrow is better 🚀,2021-04-13,set(),17,0.744001,0.256001,0.000001,1
1851,45398,i did something retarded,2021-04-13,set(),79103,0.000001,0.351001,0.649001,0


In [ ]:
#Convert timestamp to pd timestamp format EDIT: NO LONGER NEEDED
# datesConverted = []
# for index, row in data.iterrows():
#   dateInfo = row['timestamp']
#   dateInfo = dateInfo.split('/')
#   year = int(dateInfo[2])
#   day = int(dateInfo[1])
#   month = int(dateInfo[0])
#   datesConverted.append(str(year) + "-" + str(month).zfill(2) + "-" + str(day).zfill(2))
# data['timestampConvert'] = datesConverted

In [10]:
# Set the score for the data based on the class and the score.
weeks = [g for n, g in data.groupby(pd.Grouper(key='timestamp',freq='W'))]
for datafr in weeks:
  stockScores = dict()
  for index, row in datafr.iterrows():
    tickerData = (row['tickers'])
    if tickerData != 'set()':
      listOfStocks = tickerData.split('\'')[1::2]
      #print(listOfStocks)
      
      for stock in listOfStocks:
        if (stock in stockScores) :
          stockScores[stock] = stockScores[stock] + row['class'] * row['score']
        else:
          stockScores[stock] = row['class'] * row['score']
  top5stocks = dict(sorted(stockScores.items(), key=operator.itemgetter(1), reverse=True)[:5])
  print("Week of " + str(datafr['timestamp'].iloc[0]) + " " + str(top5stocks))
  #print(stocksRelevant)

Week of 2021-01-28 00:00:00 {'A': 69321, 'M': 68591, 'O': 67387, 'D': 67379, 'H': 67379}
Week of 2021-02-01 00:00:00 {'GME': 48793, 'AMC': 1932, 'A': 1747, 'LOVE': 1263, 'DD': 516}
Week of 2021-02-08 00:00:00 {'AMD': 5108, 'OCGN': 427, 'A': 228, 'AN': 141, 'VSTO': 131}
Week of 2021-02-15 00:00:00 {'GME': 11449, 'A': 3203, 'PLTR': 1461, 'K': 1399, 'V': 532}
Week of 2021-02-22 00:00:00 {'C': 12023, 'GME': 3249, 'AMC': 2274, 'LOVE': 2061, 'ALL': 2061}
Week of 2021-03-01 00:00:00 {'K': 17331, 'O': 17331, 'E': 17331, 'M': 17331, 'GME': 9370}
Week of 2021-03-08 00:00:00 {'LMAO': 38014, 'GME': 1141, 'AMC': 602, 'PLTR': 260, 'LOVE': 251}
Week of 2021-03-15 00:00:00 {'GME': 4093, 'UWMC': 1372, 'NOK': 399, 'WOW': 294, 'LOVE': 134}
Week of 2021-03-22 00:00:00 {'GME': 2697, 'MSFT': 891, 'AMC': 416, 'BB': 236, 'SEAS': 212}
Week of 2021-03-29 00:00:00 {'GME': 6693, 'AMC': 1193, 'VIAC': 185, 'TSLA': 165, 'RKT': 154}
Week of 2021-04-06 00:00:00 {'GME': 6439, 'ASO': 408, 'NIO': 286, 'VIAC': 211, 'MVIS'

In [11]:
data['length'] = [len(i.split()) for i in data['title']]

In [12]:
data['length'].min()

3

In [13]:
from sklearn.model_selection import train_test_split
sentences_train, sentences_test, y_train, y_test = train_test_split(list(data['title']), list(data['class']), train_size=0.8, random_state=1, stratify=data['class'])

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)
X_train

<1482x2015 sparse matrix of type '<class 'numpy.int64'>'
	with 8076 stored elements in Compressed Sparse Row format>

In [16]:
#baseline model 
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, list(y_train))
trainScore = classifier.score(X_train, list(y_train))
print("Train Accuracy: ", trainScore)
score = classifier.score(X_test, list(y_test))
print("Accuracy:", score)

Train Accuracy:  0.9838056680161943
Accuracy: 0.8840970350404312


In [ ]:
# nlp = spacy.load('en_core_web_sm')
# doc = nlp(testtext)
# token_list = [token for token in doc]
# filtered_tokens = [token for token in doc if not token.is_stop]
# lemmas = [f"Token: {token}, lemma: {token.lemma_}" for token in filtered_tokens]
# filtered_tokens[1].vector

In [17]:
#representing words as integers
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 in
print(sentences_train[2])
print(X_train[2])

welcome to the endgame.
[191, 5, 2, 896]


In [ ]:
#adding padding so that the integer sequences have the same length
from keras.preprocessing.sequence import pad_sequences

maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
print(X_train[0, :])

[  9 895  77 112 290  69   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [ ]:
#simple model

# from keras.models import Sequential
# from keras import layers


# #size of vector
# embedding_dim = 70
# model = Sequential()
# #converting text into the form of vectors
# model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen))
# model.add(layers.Flatten())
# model.add(layers.Dense(20, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid')) #predict only one label
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.summary()

In [ ]:
y_train=np.array(y_train).astype('float64')
y_test=np.array(y_test).astype('float64')

In [ ]:
# #plotting function
# import matplotlib.pyplot as plt
# plt.style.use('ggplot')

# def plot_history(history):
#     acc = history.history['acc']
#     val_acc = history.history['val_acc']
#     loss = history.history['loss']
#     val_loss = history.history['val_loss']
#     x = range(1, len(acc) + 1)

#     plt.figure(figsize=(12, 5))
#     plt.subplot(1, 2, 1)
#     plt.plot(x, acc, 'b', label='Training acc')
#     plt.plot(x, val_acc, 'r', label='Validation acc')
#     plt.title('Training and validation accuracy')
#     plt.legend()
#     plt.subplot(1, 2, 2)
#     plt.plot(x, loss, 'b', label='Training loss')
#     plt.plot(x, val_loss, 'r', label='Validation loss')
#     plt.title('Training and validation loss')
#     plt.legend()

In [ ]:
# history = model.fit(X_train, y_train, epochs=20, verbose=False, validation_data=(X_test, y_test), batch_size=10)
# loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
# print("Training Accuracy: {:.4f}".format(accuracy))
# loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
# print("Testing Accuracy:  {:.4f}".format(accuracy))
# # plot_history(history)

In [ ]:
#CNN model

from keras.models import Sequential
from keras import layers

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 50)           116900    
_________________________________________________________________
conv1d (Conv1D)              (None, 96, 128)           32128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 157,349
Trainable params: 157,349
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/20
149/149 [==============================] - 3s 13ms/step - loss: 0.6539 - accuracy: 0.6120 - val_loss: 0.3886 - val_accuracy: 0.8464
Epoch 2/20
149/149 [==============================] - 2s 12ms/step - loss: 0.2098 - accuracy: 0.9149 - val_loss: 0.3007 - val_accuracy: 0.8895
Epoch 3/20
149/149 [==============================] - 2s 11ms/step - loss: 0.0413 - accuracy: 0.9869 - val_loss: 0.3135 - val_accuracy: 0.9003
Epoch 4/20
149/149 [==============================] - 2s 12ms/step - loss: 0.0198 - accuracy: 0.9956 - val_loss: 0.3575 - val_accuracy: 0.8922
Epoch 5/20
149/149 [==============================] - 2s 11ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.3975 - val_accuracy: 0.8949
Epoch 6/20
149/149 [==============================] - 2s 11ms/step - loss: 9.0627e-04 - accuracy: 1.0000 - val_loss: 0.4169 - val_accuracy: 0.8976
Epoch 7/20
149/149 [==============================] - 2s 12ms/step - loss: 4.8202e-04 - accuracy: 1.0000 - val_loss: 0.4299 - val_accuracy

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions.shape

(371, 1)

In [ ]:
threshold = 0.5
predictions = (predictions >= threshold).astype(int)

In [ ]:
df = pd.DataFrame({'text': list(sentences_test), 'sentiment': list(predictions)})

In [ ]:
df

,text,sentiment
0,i love all of you.,[1]
1,amc yolo update.. ape love movies 🦍🦍🦍,[1]
2,"my own ""if gme hits.."" promise that i hope i f...",[1]
3,i like the stock and free tendies,[1]
4,gme the high risk high reward play,[0]
...,...,...
366,a bit delayed here in oz but i'm fucking in yo...,[0]
367,32 gme shares at $250 a share,[1]
368,nok isn't banned 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀,[1]
369,dont let the vultures win on amc.,[1]


In [ ]:
df['label'] = df['sentiment'].apply(lambda x: 'positive' if x == [1] else 'negative')

In [ ]:
df

,text,sentiment,label
0,i love all of you.,[1],positive
1,amc yolo update.. ape love movies 🦍🦍🦍,[1],positive
2,"my own ""if gme hits.."" promise that i hope i f...",[1],positive
3,i like the stock and free tendies,[1],positive
4,gme the high risk high reward play,[0],negative
...,...,...,...
366,a bit delayed here in oz but i'm fucking in yo...,[0],negative
367,32 gme shares at $250 a share,[1],positive
368,nok isn't banned 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀,[1],positive
369,dont let the vultures win on amc.,[1],positive


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, predictions)

0.894878706199461

In [ ]:
#RNN + TRANSFORMER
